In [43]:
# install pymysql
!pip install pymysql

# Import pymysql and pandas
import pymysql
import pandas as pd

try:
    # Establish connection
    conn = pymysql.connect(
        host='localhost',
        user='root',         # Your MySQL username
        password='Ankita@123',         # Your MySQL password
        database='titanic',   # make sure the database in loaded in your(MySQL)
        port=3306
    )

    # Load data into DataFrame
    query = "SELECT * FROM titanic"  # Replace with your table
    df = pd.read_sql(query, conn)

    # Display results
    print("Connection successful! Here's your data:")
    display(df.head())

except pymysql.Error as e:
    print(f"MySQL Error: {e}")

finally:
    if 'conn' in locals():
        conn.close()
        print("Connection closed")


Connection successful! Here's your data:


C:\Users\Acer\AppData\Local\Temp\ipykernel_11956\3617836051.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,,S


Connection closed


In [44]:
# Import libraries
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import pickle


In [47]:
df = sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [49]:
# Basic nfo of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [51]:
# Is any null value present in the daataset
print("Null values per column:")
print(df.isnull().sum())

Null values per column:
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [53]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [55]:
# Label encoding
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])
df['embarked'] = le.fit_transform(df['embarked'].astype(str))

In [59]:
# Featuring x and y
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
X = df[features]
y = df['survived']

In [61]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

In [63]:
# DMatrix is XGBoost’s optimized data format (handles missing values efficiently)
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [65]:
# Setting XGBoost Parameters

params = {
    'objective': 'binary:logistic',  # Binary classification
    'eval_metric': 'logloss',       # Evaluation metric
    'eta': 0.1,                     # Learning rate
    'max_depth': 4,                 # Max depth of trees
    'subsample': 0.8,               # % of training data used per tree
    'colsample_bytree': 0.8,        # % of features used per tree
    'seed': 42,                     # Random seed
    'tree_method': 'hist',          # Efficient handling of missing values
    'missing': float('nan')         # Treat NaN as missing (default)
}


In [67]:
# model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=[(dtrain, 'train'), (dtest, 'eval')],
    early_stopping_rounds=10,
    verbose_eval=10
)


[0]	train-logloss:0.63369	eval-logloss:0.60870
[10]	train-logloss:0.48929	eval-logloss:0.46478
[20]	train-logloss:0.40388	eval-logloss:0.38172
[30]	train-logloss:0.37175	eval-logloss:0.36040
[40]	train-logloss:0.34993	eval-logloss:0.34531
[50]	train-logloss:0.33165	eval-logloss:0.34223
[56]	train-logloss:0.32249	eval-logloss:0.34269


C:\Users\Acer\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [16:58:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "missing" } are not used.

  self.starting_round = model.num_boosted_rounds()


In [69]:
# Predict the model
y_pred_prob = model.predict(dtest)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]


In [71]:

# Evaluation
print("\nModel Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Model Evaluation:
Accuracy: 0.8492

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.89      0.89       118
           1       0.78      0.77      0.78        61

    accuracy                           0.85       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.85      0.85      0.85       179



In [73]:

# Save model
with open("xgboost.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model saved as 'xgboost.pkl'")

✅ Model saved as 'xgboost.pkl'
